In [278]:
import numpy as np
import pandas as pd

In [279]:
# Importing all tables

customer = pd.read_csv('customers.csv')
ticket = pd.read_csv('support_tickets.csv')
campaign = pd.read_csv('campaigns.csv')
review = pd.read_csv('customer_reviews_complete.csv')
transaction = pd.read_csv('transactions.csv')
interaction = pd.read_csv('interactions.csv')

## Cleaning customer table

In [280]:
customer.sample()

,customer_id,full_name,age,gender,email,phone,street_address,city,state,zip_code,registration_date,preferred_channel
2854,abf6432a-0d1f-4d9d-ad5e-6790d03b0800,Derek Carrillo,49.0,Male,derek.carrillo@gmail.com,266.901.9461x16081,19731 Cox Falls Suite 542,Miami,Florida,42915.0,9/11/2023,online


In [281]:
customer.shape

(5000, 12)

In [282]:
customer.isnull().sum()

customer_id            0
full_name            106
age                  186
gender               112
email                111
phone                186
street_address       177
city                  97
state                 93
zip_code              96
registration_date      0
preferred_channel    114
dtype: int64

In [283]:
customer.duplicated().sum()

np.int64(0)

In [284]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        5000 non-null   object 
 1   full_name          4894 non-null   object 
 2   age                4814 non-null   float64
 3   gender             4888 non-null   object 
 4   email              4889 non-null   object 
 5   phone              4814 non-null   object 
 6   street_address     4823 non-null   object 
 7   city               4903 non-null   object 
 8   state              4907 non-null   object 
 9   zip_code           4904 non-null   float64
 10  registration_date  5000 non-null   object 
 11  preferred_channel  4886 non-null   object 
dtypes: float64(2), object(10)
memory usage: 468.9+ KB


In [285]:
customer.drop(columns=['email','phone','zip_code','street_address'],inplace=True)

In [286]:
customer['age'] = pd.to_numeric(customer['age'],errors='coerce').astype('Int64')
customer['registration_date'] = pd.to_datetime(customer['registration_date'],errors='coerce')

In [287]:
customer['full_name'] = customer['full_name'].fillna('Unknown')
customer['gender'] = customer['gender'].fillna('Unknown')
customer['city'] = customer['city'].fillna('Unknown')
customer['state'] = customer['state'].fillna('Unknown')
customer['preferred_channel'] = customer['preferred_channel'].fillna('Unknown')

In [288]:
customer['age'] = customer['age'].fillna(customer['age'].median())

In [289]:
customer['year'] = customer["registration_date"].dt.year

In [290]:
customer['age_group'] = pd.cut(
    customer['age'],
    bins=[1,18, 35, 45, 60, 100],
    labels=['1-17','18-35', '36-45', '46-60', '60+'],
    right=True
)

## Cleaning support_ticket table

In [291]:
ticket.sample()

,ticket_id,customer_id,issue_category,priority,submission_date,resolution_date,resolution_status,resolution_time_hours,customer_satisfaction_score,notes
2868,3e889704-5e91-49be-bf9d-8875498cda74,ff9cc2d8-db23-42f4-a885-0b17c5075f6b,Account Issue,Low,12/13/2024 0:00,12/14/2024 6:00,Resolved,30.0,4.0,Customer unable to update payment information ...


In [292]:
ticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ticket_id                    3000 non-null   object 
 1   customer_id                  3000 non-null   object 
 2   issue_category               2929 non-null   object 
 3   priority                     2940 non-null   object 
 4   submission_date              3000 non-null   object 
 5   resolution_date              2756 non-null   object 
 6   resolution_status            2943 non-null   object 
 7   resolution_time_hours        2701 non-null   float64
 8   customer_satisfaction_score  2659 non-null   float64
 9   notes                        2939 non-null   object 
dtypes: float64(2), object(8)
memory usage: 234.5+ KB


In [293]:
ticket.isnull().sum()

ticket_id                        0
customer_id                      0
issue_category                  71
priority                        60
submission_date                  0
resolution_date                244
resolution_status               57
resolution_time_hours          299
customer_satisfaction_score    341
notes                           61
dtype: int64

In [294]:
ticket.duplicated().sum()

np.int64(0)

In [295]:
ticket.shape

(3000, 10)

In [296]:
ticket['submission_date'] = pd.to_datetime(ticket['submission_date'],errors='coerce')
ticket['resolution_date'] = pd.to_datetime(ticket['resolution_date'],errors='coerce')

In [297]:
ticket.drop(columns=['notes'],inplace=True)

In [298]:
ticket['issue_category'] = ticket['issue_category'].fillna('Unknown')
ticket['priority'] = ticket['priority'].fillna('Unknown')

In [299]:
# If resolution_date exists → status must be resolved
ticket.loc[
    ticket['resolution_status'].isna() & ticket['resolution_date'].notna(),
    'resolution_status'
] = 'resolved'

# If resolution_date is null → status must be open
ticket.loc[
    ticket['resolution_status'].isna() & ticket['resolution_date'].isna(),
    'resolution_status'
] = 'open'

In [300]:
ticket.loc[
    ticket['resolution_time_hours'].isna(),
    'resolution_time_hours'
] = (
    (ticket['resolution_date'] - ticket['submission_date'])
    .dt.total_seconds() / 3600
)

In [301]:
avg_score = ticket['customer_satisfaction_score'].mean()
ticket['customer_satisfaction_score'] = ticket['customer_satisfaction_score'].fillna(round(avg_score))

In [302]:
ticket['ticket_year'] = ticket['submission_date'].dt.year
ticket['ticket_month'] = ticket['submission_date'].dt.month
ticket['ticket_day'] = ticket['submission_date'].dt.day

In [303]:
ticket['resolution_speed'] = pd.cut(
    ticket['resolution_time_hours'],
    bins=[0, 24, 72, 168, float('inf')],
    labels=['within_1_day', '1_3_days', '3_7_days', 'more_than_7_days']
)

In [304]:
ticket['isResolved'] = ticket['resolution_date'].notna().astype(int)

In [305]:
ticket['high_priority_flag'] = (ticket['priority'] == 'high').astype(int)
ticket['sla_breach'] = (ticket['resolution_time_hours'] > 72).astype(int)

## Proceding with review table

In [306]:
review.sample()

,review_id,customer_id,product_name,product_category,full_name,transaction_date,review_date,rating,review_title,review_text
114,rev_000115,4795418a-acf3-44b2-805a-7b8c4545f04b,Vizio SmartCast TV,TVs,Brenda Harper,4/24/2024,5/13/2024,4,A Smart Choice with a Few Glitches,"I recently purchased the Vizio SmartCast TV, a..."


In [307]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         1000 non-null   object
 1   customer_id       1000 non-null   object
 2   product_name      976 non-null    object
 3   product_category  981 non-null    object
 4   full_name         981 non-null    object
 5   transaction_date  1000 non-null   object
 6   review_date       1000 non-null   object
 7   rating            1000 non-null   int64 
 8   review_title      1000 non-null   object
 9   review_text       1000 non-null   object
dtypes: int64(1), object(9)
memory usage: 78.3+ KB


In [308]:
review.isnull().sum()

review_id            0
customer_id          0
product_name        24
product_category    19
full_name           19
transaction_date     0
review_date          0
rating               0
review_title         0
review_text          0
dtype: int64

In [309]:
review.shape

(1000, 10)

In [310]:
review.duplicated().sum()

np.int64(0)

In [311]:
review['transaction_date'] = pd.to_datetime(review['transaction_date'],errors='coerce')
review['review_date'] = pd.to_datetime(review['review_date'],errors='coerce')

In [312]:
review = review.merge(
    customer[['customer_id','full_name']],
    how='left',
    on='customer_id',
    suffixes=('','_cust')
)

review['full_name'] = review['full_name'].fillna(review['full_name_cust'])
review['full_name'] = review['full_name'].fillna('Unknown')

review.drop(columns=['full_name_cust'],inplace=True)

In [313]:
review['product_name'] = review['product_name'].fillna(
    review['product_name'].mode()[0]
)

review['product_category'] = review['product_category'].fillna(
    review['product_category'].mode()[0]
)


In [314]:
review['review_year'] = review['review_date'].dt.year
review['review_month'] = review['review_date'].dt.month
review['days_after_transaction'] = (review['review_date'] - review['transaction_date']).dt.days

In [315]:
review['sentiment'] = pd.cut(
    review['rating'],
    bins=[0,2,3,5],
    labels=['Negative','Neutral','Positive']
)

## Proceed with campaign table

In [316]:
campaign.sample()

,campaign_id,campaign_name,campaign_type,start_date,end_date,target_segment,budget,impressions,clicks,conversions,conversion_rate,roi
101,736327b2-acd1-4006-b381-7475c5c1d908,VIP Bonanza 2022,TV Advertisement,8/22/2022,11/20/2022,Midwest,124039.19,4935110.0,5868.0,251.0,4.28,-0.2


In [317]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   campaign_id      200 non-null    object 
 1   campaign_name    194 non-null    object 
 2   campaign_type    194 non-null    object 
 3   start_date       200 non-null    object 
 4   end_date         200 non-null    object 
 5   target_segment   200 non-null    object 
 6   budget           196 non-null    float64
 7   impressions      196 non-null    float64
 8   clicks           190 non-null    float64
 9   conversions      193 non-null    float64
 10  conversion_rate  198 non-null    float64
 11  roi              197 non-null    float64
dtypes: float64(6), object(6)
memory usage: 18.9+ KB


In [318]:
campaign['start_date'] = pd.to_datetime(campaign['start_date'],errors='coerce')
campaign['end_date'] = pd.to_datetime(campaign['end_date'],errors='coerce')

In [319]:
campaign.isnull().sum()

campaign_id         0
campaign_name       6
campaign_type       6
start_date          0
end_date            0
target_segment      0
budget              4
impressions         4
clicks             10
conversions         7
conversion_rate     2
roi                 3
dtype: int64

In [320]:
campaign.shape

(200, 12)

In [321]:
campaign['campaign_name'] = campaign['campaign_name'].fillna('Unknown')
campaign['campaign_type'] = campaign['campaign_type'].fillna('Unknown')

In [322]:
campaign['budget'] = campaign['budget'].fillna(0)
campaign['impressions'] = campaign['impressions'].fillna(0)
campaign['clicks'] = campaign[['clicks', 'impressions']].min(axis=1)
campaign['conversions'] = campaign[['conversions', 'clicks']].min(axis=1)


In [323]:
campaign['conversion_rate'] = (
    campaign['conversions'] / campaign['clicks']
).replace([float('inf'), -float('inf')], 0).fillna(0)

campaign['roi'] = (
    campaign['conversions'] / campaign['budget']
).replace([float('inf'), -float('inf')], 0).fillna(0)

In [324]:
campaign['campaign_duration_days'] = ( campaign['end_date'] - campaign['start_date'] ).dt.days

In [325]:
campaign['start_year'] = campaign['start_date'].dt.year
campaign['start_month'] = campaign['start_date'].dt.month

In [326]:
campaign['cost_per_click'] = (
    campaign['budget'] / campaign['clicks']
).replace([float('inf'), -float('inf')], 0).fillna(0)

campaign['cost_per_conversion'] = (
    campaign['budget'] / campaign['conversions']
).replace([float('inf'), -float('inf')], 0).fillna(0)

## Proceed with transaction Table

In [327]:
transaction.sample()

,transaction_id,customer_id,product_name,product_category,quantity,price,transaction_date,store_location,payment_method,discount_applied
20724,b9119af4-76a5-4010-bdd9-cf6ffa3596cc,275b6129-29db-4520-adbb-676c9c8c0b3c,Duvet Cover,Bedding,1.0,114.7,11/3/2024,Online,Debit Card,0.0


In [328]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32295 entries, 0 to 32294
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    32295 non-null  object 
 1   customer_id       32295 non-null  object 
 2   product_name      31617 non-null  object 
 3   product_category  31609 non-null  object 
 4   quantity          31651 non-null  float64
 5   price             31673 non-null  float64
 6   transaction_date  32295 non-null  object 
 7   store_location    31651 non-null  object 
 8   payment_method    31635 non-null  object 
 9   discount_applied  31684 non-null  float64
dtypes: float64(3), object(7)
memory usage: 2.5+ MB


In [329]:
transaction.isnull().sum()

transaction_id        0
customer_id           0
product_name        678
product_category    686
quantity            644
price               622
transaction_date      0
store_location      644
payment_method      660
discount_applied    611
dtype: int64

In [330]:
transaction.shape

(32295, 10)

In [331]:
transaction['transaction_date'] = pd.to_datetime(transaction['transaction_date'],errors='coerce')

In [332]:
transaction['product_name'] = transaction['product_name'].fillna(
    transaction['product_name'].mode()[0]
)

transaction['product_category'] = transaction['product_category'].fillna(
    transaction['product_category'].mode()[0]
)

In [333]:
transaction['quantity'] = transaction['quantity'].fillna(
    transaction.groupby('product_name')['quantity'].transform('median')
)

transaction['quantity'] = transaction['quantity'].fillna(
    transaction.groupby('product_category')['quantity'].transform('median')
)

transaction['quantity'] = transaction['quantity'].fillna(
    transaction['quantity'].median())


In [334]:
transaction['price'] = transaction['price'].fillna(
    transaction.groupby('product_name')['price'].transform('median')
)

transaction['price'] = transaction['price'].fillna(
    transaction.groupby('product_category')['price'].transform('median')
)

transaction['price'] = transaction['price'].fillna(
    transaction['price'].median()
)

In [335]:
transaction['store_location'] = transaction['store_location'].fillna(
    transaction['store_location'].mode()[0]
)

transaction['payment_method'] = transaction['payment_method'].fillna(
    transaction['payment_method'].mode()[0])

In [336]:
transaction['discount_applied'] = transaction['discount_applied'].fillna(
    transaction.groupby('product_category')['discount_applied'].transform('median')
)
transaction['discount_applied'] = transaction['discount_applied'].fillna(
    transaction['discount_applied'].median()
)

In [337]:
transaction['txn_year'] = transaction['transaction_date'].dt.year
transaction['txn_month'] = transaction['transaction_date'].dt.month
transaction['txn_weekday'] = transaction['transaction_date'].dt.day_name()

In [338]:
transaction['gross_amount'] = transaction['price']*transaction['quantity']
transaction['final_amount'] = transaction['gross_amount'] - (transaction['gross_amount']*transaction['discount_applied']/100)

### Add some more features to customer table based on transaction table

In [339]:
customer_spend = (
    transaction.groupby('customer_id')['final_amount']
    .sum()
    .reset_index(name='total_spent')
)

customer = customer.merge(customer_spend, on='customer_id', how='left')
customer['total_spent'] = customer['total_spent'].fillna(0)

In [340]:
customer['spending_category'] = pd.qcut(
    customer['total_spent'],
    q=3,
    labels=['Low','Medium','High']
)

In [341]:
customer_txn_count = (
    transaction.groupby('customer_id')['transaction_id']
    .count()
    .reset_index(name='total_transactions')
)
customer = customer.merge(customer_txn_count, on='customer_id', how='left')
customer['total_transactions'] = customer['total_transactions'].fillna(0)


## Proceed with Interaction table

In [342]:
interaction.sample()

,interaction_id,customer_id,channel,interaction_type,interaction_date,duration,page_or_product,session_id
61020,5bec2f6d-7558-46c9-8a36-9c6cf6e5631e,9fee71c0-7d17-456d-a61d-b191da606827,in-store kiosk,product_lookup,4/20/2024 19:29,49.0,Dell XPS 15,9fee71c0-7d17-456d-a61d-b191da606827_session_1


In [343]:
interaction.shape

(100000, 8)

In [344]:
interaction.isnull().sum()

interaction_id         0
customer_id            0
channel             2002
interaction_type    2022
interaction_date       0
duration            1963
page_or_product     1927
session_id             0
dtype: int64

In [345]:
interaction.duplicated().sum()

np.int64(0)

In [346]:
interaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   interaction_id    100000 non-null  object 
 1   customer_id       100000 non-null  object 
 2   channel           97998 non-null   object 
 3   interaction_type  97978 non-null   object 
 4   interaction_date  100000 non-null  object 
 5   duration          98037 non-null   float64
 6   page_or_product   98073 non-null   object 
 7   session_id        100000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 6.1+ MB


In [347]:
interaction['interaction_date'] = pd.to_datetime(interaction['interaction_date'],errors='coerce')

In [348]:
interaction['channel'] = interaction['channel'].fillna(
    interaction['channel'].mode()[0]
)
interaction['interaction_type'] = interaction['interaction_type'].fillna(
    interaction['interaction_type'].mode()[0]
)
interaction['duration'] = interaction['duration'].fillna(
    interaction.groupby('interaction_type')['duration'].transform('median')
)
interaction['page_or_product'] = interaction['page_or_product'].fillna(
    interaction['page_or_product'].mode()[0]
)

In [349]:
interaction['int_year'] = interaction['interaction_date'].dt.year
interaction['int_month'] = interaction['interaction_date'].dt.month
interaction['int_weekday'] = interaction['interaction_date'].dt.day_name()
interaction['int_time'] = interaction['interaction_date'].dt.time

In [350]:
interaction['time_period'] = pd.cut(
    interaction['interaction_date'].dt.hour,
    bins=[0,6,12,18,24],
    labels=['Night','Morning','Afternoon','Evening'],
    right=False
)

### Add some more features to customer table based on transaction table

In [351]:
fav_page = (
    interaction.groupby(['customer_id','page_or_product'])
    .size()
    .reset_index(name='count')
    .sort_values(['customer_id','count'], ascending=[True, False])
    .drop_duplicates('customer_id')
    [['customer_id','page_or_product']]
)

customer = customer.merge(fav_page, on='customer_id', how='left')
customer.rename(columns={'page_or_product':'favorite_page_or_product'}, inplace=True)

customer['favorite_page_or_product'] = customer['favorite_page_or_product'].fillna('Unknown')